# Deep Learning for Predictive Maintenance

Deep learning has proven to show superior performance in certain domains such as object recognition and image classification. It has also gained popularity in  domains such as finance where time-series data plays an important role. Predictive Maintenance is also a domain where data is collected over time to monitor the state of an asset with the goal of finding patterns to predict failures which can also benefit from certain deep learning algorithms. Among the deep learning methods, Long Short Term Memory [(LSTM)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) networks are especially appealing to the predictive maintenance domain due to the fact that they are very good at learning from sequences. This fact lends itself to their applications using time series data by making it possible to look back for longer periods of time to detect failure patterns. In this notebook, we build an LSTM network for the data set and scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict remaining useful life of aircraft engines. In summary, the template uses simulated aircraft sensor values to predict when an aircraft engine will fail in the future so that maintenance can be planned in advance. 

This notebook serves as a tutorial for beginners looking to apply deep learning in predictive maintenance domain and uses a simple scenario where only one data source (sensor values) is used to make predictions. In more advanced predictive maintenance scenarios such as in [Predictive Maintenance Modelling Guide](https://gallery.cortanaintelligence.com/Notebook/Predictive-Maintenance-Modelling-Guide-R-Notebook-1), there are many other data sources (i.e. historical maintenance records, error logs, machine and operator features etc.) which may require different types of treatments to be used in the deep learning networks. Since predictive maintenance is not a typical domain for deep learning, its application is an open area of research. 

This notebook uses [keras](https://keras.io/) deep learning library.

## Problem Description
To predict remaining useful life (or time to failure) of aircraft engines <a href="https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan">[3]</a> based on scenario described at <a href="https://github.com/Azure/lstms_for_predictive_maintenance/blob/master/Deep%20Learning%20Basics%20for%20Predictive%20Maintenance.ipynb">[1]</a> and <a href="https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-2A-of-3-train-and-evaluate-regression-models-2">[2]</a>.
The network uses simulated aircraft sensor values to predict when an aircraft engine will fail in the future so that maintenance can be planned in advance.
The question to ask is "Given these aircraft engine operation and failure events history, can we predict when an in-service engine will fail?"
We re-formulate this question into two closely relevant questions and answer them using two different types of machine learning models:

- Binary classification: Is this engine going to fail within w1 cycles? (in this tutorial)
- Regression models: How many more cycles an in-service engine will last before it fails? (as homework)

## Data
In the **Dataset** there are the training, test and ground truth datasets.
The training data consists of **multiple multivariate time series** with "cycle" as the time unit, together with 21 sensor readings for each cycle.
Each time series can be assumed as being generated from a different engine of the same type.
The testing data has the same data schema as the training data.
The only difference is that the data does not indicate when the failure occurs.
Finally, the ground truth data provides the number of remaining working cycles for the engines in the testing data.


You can find more details about the data at <a href="https://github.com/Azure/lstms_for_predictive_maintenance/blob/master/Deep%20Learning%20Basics%20for%20Predictive%20Maintenance.ipynb">[1]</a> and <a href="https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-2A-of-3-train-and-evaluate-regression-models-2">[2]</a>.

In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline

## Data Ingestion
In the following section, we ingest the training, test and ground truth datasets from azure storage. The training data consists of multiple multivariate time series with "cycle" as the time unit, together with 21 sensor readings for each cycle. Each time series can be assumed as being generated from a different engine of the same type. The testing data has the same data schema as the training data. The only difference is that the data does not indicate when the failure occurs. Finally, the ground truth data provides the number of remaining working cycles for the engines in the testing data. You can find more details about the type of data used for this notebook at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3).

In [3]:
import urllib
import os

basedataurl = "http://azuremlsamples.azureml.net/templatedata/"
datafile = "PM_train.txt"
# Download the file once, and only once.
if not os.path.isfile(datafile): urllib.request.urlretrieve(basedataurl+datafile, datafile)

datafile = "PM_test.txt"
# Download the file once, and only once.
if not os.path.isfile(datafile): urllib.request.urlretrieve(basedataurl+datafile, datafile)

datafile = "PM_truth.txt"
# Download the file once, and only once.
if not os.path.isfile(datafile): urllib.request.urlretrieve(basedataurl+datafile, datafile)



In [4]:
# read training data 
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [5]:
# read test data
test_df = pd.read_csv('PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']

In [6]:
# read ground truth data
truth_df = pd.read_csv('PM_truth.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [7]:
train_df = train_df.sort_values(['id','cycle'])
display(train_df.head())

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Data Preprocessing
First step is to generate labels for the training data which are Remaining Useful Life (RUL), label1 and label2 as was done in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3). Here, we will only make use of "label1" for binary clasification, while trying to answer the question: is a specific engine going to fail within w1 cycles?

In [8]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
display(train_df.head())

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [9]:
# generate label columns for training data
w1 = 30
w0 = 15
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df['label2'] = train_df['label1']
train_df.loc[train_df['RUL'] <= w0, 'label2'] = 2
display(train_df.head())

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0,0


In the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) , cycle column is also used for training so we will also include the cycle column. Here, we normalize the columns in the training data.

In [10]:
# MinMax normalization
from sklearn.externals import joblib
scaler_filename = "scaler.pkl"

train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1','label2'])
min_max_scaler = preprocessing.MinMaxScaler()

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)

joblib.dump(min_max_scaler, scaler_filename) 

join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.01108


In [11]:
cols_normalize

Index(['cycle_norm', 's1', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       's17', 's18', 's19', 's2', 's20', 's21', 's3', 's4', 's5', 's6', 's7',
       's8', 's9', 'setting1', 'setting2', 'setting3'],
      dtype='object')

Next, we prepare the test data. We first normalize the test data using the parameters from the MinMax normalization applied on the training data.

In [12]:
min_max_scaler = joblib.load(scaler_filename) 
    
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df_original = test_df.copy()
test_df.head()


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108


Next, we use the ground truth dataset to generate labels for the test data.

In [13]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [14]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
display(test_df.head())

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm,RUL
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138


In [15]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df['label2'] = test_df['label1']
test_df.loc[test_df['RUL'] <= w0, 'label2'] = 2
display(test_df.head())

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108,138,0,0


In the rest of the notebook, we train an LSTM network that we will compare to the results in [Predictive Maintenance Template Step 2B of 3](https://gallery.cortanaintelligence.com/Experiment/Predictive-Maintenance-Step-2B-of-3-train-and-evaluate-binary-classification-models-2) where a series of machine learning models are used to train and evaluate the binary classification model that uses column "label1" as the label.

## Modelling

The traditional predictive maintenance machine learning models are based on feature engineering which is manual construction of right features using domain expertise and similar methods. This usually makes these models hard to reuse since feature engineering is specific to the problem scenario and the available data which varies from one business to the other. Perhaps the most attractive part of applying deep learning in the predictive maintenance domain is the fact that these networks can automatically extract the right features from the data, eliminating the need for manual feature engineering.

When using LSTMs in the time-series domain, one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) which are rolling mean and rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

One critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. For example, computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing and abstracting of values over such a long period, instead, using all 50 values as input may provide better results. While feature engineering over large window sizes may not make sense, LSTMs are able to use larger window sizes and use all the information in the window as input. Below, we illustrate the approach.

In [16]:
import azureml
import azureml.core

In [17]:
SUBSCRIPTION_ID = '<ENTER HERE YOUR AZURE SUBSCRIPTION>'
WORKSPACE_NAME = 'jptr_predictive_maintenance_ws'
RESOURCE_GROUP = 'jptr_predictive_maintenance_rg'
WORKSPACE_REGION = 'westus2'

In [18]:
from azureml.core import Workspace

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.33


In [19]:
from azureml.core import Workspace

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

ws = Workspace.create(name = WORKSPACE_NAME,
                      subscription_id = SUBSCRIPTION_ID,
                      resource_group = RESOURCE_GROUP, 
                      location = WORKSPACE_REGION,
                      exist_ok=True)


SDK version: 1.0.33
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FYPGPSLLU to authenticate.
Interactive authentication successfully completed.


Deploying KeyVault with name jptrpredkeyvaultebe8583d.
Deploying StorageAccount with name jptrpredstorage42356bd6e.
Deploying AppInsights with name jptrpredinsights87770a54.
Deployed AppInsights with name jptrpredinsights87770a54.
Deployed StorageAccount with name jptrpredstorage42356bd6e.
Deployed KeyVault with name jptrpredkeyvaultebe8583d.
Deploying Workspace with name jptr_predictive_maintenance_ws.
Deployed Workspace with name jptr_predictive_maintenance_ws.


In [20]:
# Save the workspace config
ws.write_config()

# Reconnect to the workspace (if you're not already signed in, you'll be prompted to authenticate with a code as before)
ws = Workspace.from_config()

In [22]:
%%sh
pwd

/home/nbuser/library


### We Start the experiment and tracking the RUN

In [23]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import os

model_name = "predictivemodel.h5"
model_dbfs = os.path.join("/home/nbuser/library/", model_name)
run_history_name = 'predictive-maintenance-local'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "Predictive_Maintenance")


In [26]:
%%sh
ls

Good Deploy Predictive Maintenance.ipynb
PM_test.txt
PM_train.txt
PM_truth.txt
README.md
scaler.pkl


[Keras LSTM](https://keras.io/layers/recurrent/) layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step.

In [27]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
        
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]
  
def Build_Network(lstm_1, lstm_2, dropout, epochs, sequence_length, nb_features, nb_out, save_weights_filepath ):
    model = Sequential()

    model.add(LSTM(
             input_shape=(sequence_length, nb_features),
             units=lstm_1,
             return_sequences=True))
    model.add(Dropout(dropout))

    model.add(LSTM(
              units=lstm_2,
              return_sequences=False))
    model.add(Dropout(dropout))

    model.add(Dense(units=nb_out, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #print(model.summary())
    
    history = model.fit(seq_array, label_array, epochs=epochs, batch_size=200, validation_split=0.05, verbose=0,
              callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto'),
                           keras.callbacks.ModelCheckpoint(save_weights_filepath, 
                                                         monitor='val_acc', 
                                                         verbose=0, save_best_only=True, mode='max')])
    
    model.load_weights(save_weights_filepath)
    
    return model, history


In [28]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

In [29]:
%%time
root_run = myexperiment.start_logging()

# pick a large window size of 50 cycles
sequence_lengths = [10,25,50,70]
LSTM_1 = [50,100,150]
LSTM_2 = [20,50,100]
DROPOUT = [0.1,0.2,0.5]
EPOCHS = [10,20,40,50]

save_weights_filepath = "best-weights.h5" 
LSTM_1 = 100
LSTM_2 = 50
DROPOUT = 0.2
EPOCHS = 10


i = 0
for sequence_length in sequence_lengths:
  
    with root_run.child_run("seq-length-" + str(sequence_length)) as run:
      
        print("Runing Child:", i )
        print("Using sequence_length=", sequence_length)
        i = i + 1
        # generator for the sequences
        seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
                   for id in train_df['id'].unique())

        # generate sequences and convert to numpy array
        seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
        seq_array.shape

        # generate labels
        label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
                     for id in train_df['id'].unique()]
        label_array = np.concatenate(label_gen).astype(np.float32)
        label_array.shape

        #### LSTM NETWORK #####
        nb_features = seq_array.shape[2]
        nb_out = label_array.shape[1]

        model, history = Build_Network(LSTM_1, LSTM_2, DROPOUT, EPOCHS, sequence_length, nb_features, nb_out, save_weights_filepath )

        # training metrics
        scores = model.evaluate(seq_array, label_array, verbose=0, batch_size=200)
        print('Accurracy: {}'.format(scores[1]))

        # make predictions and compute confusion matrix
        y_pred = model.predict_classes(seq_array,verbose=1, batch_size=200)
        y_true = label_array
        print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
        cm = confusion_matrix(y_true, y_pred)
        print(cm)

        # compute precision and recall on Training data
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        print( 'training_precision = ', precision, '\n', 'training_recall = ', recall)

        # Transforming Test Set
        seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                           for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

        seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
        seq_array_test_last.shape

        y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

        label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
        label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
        label_array_test_last.shape

        print(seq_array_test_last.shape)
        print(label_array_test_last.shape)

        # test metrics
        scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=0)
        print('Accurracy: {}'.format(scores_test[1]))

        # make predictions and compute confusion matrix
        y_pred_test = model.predict_classes(seq_array_test_last)
        y_true_test = label_array_test_last
        print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
        cm = confusion_matrix(y_true_test, y_pred_test)
        print(cm)

        # compute precision and recall
        precision_test = precision_score(y_true_test, y_pred_test)
        recall_test = recall_score(y_true_test, y_pred_test)
        f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
        print( 'Test Precision: ', precision_test, '\n', 'Test Recall: ', recall_test,'\n', 'F1-score:', f1_test )

        #Save model on this child_run to Azure ML Experiment
        model.save(model_name)
        run.upload_file("outputs/"+model_name, model_name)        
        
        # Log Model design variables
        run.log("sequence_length", sequence_length)
        run.log("Epochs",EPOCHS)
        run.log("LSTM_1", LSTM_1)
        run.log("LSTM_2", LSTM_2)
        run.log("DROPOUT", DROPOUT)
        #Result Metrics
        run.log("Test_Precision", precision_test)
        run.log("Test_Recall", recall_test)
        run.log("Test_F1Score", f1_test)
        
        
             
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Runing Child: 0
Using sequence_length= 10
Accurracy: 0.9635780205589871
19631/19631 [==============================] - 4s 211us/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels
[[16283   248]
 [  467  2633]]
training_precision =  0.9139187782020132 
 training_recall =  0.8493548387096774
(100, 10, 25)
(100, 1)
Accurracy: 0.93
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels
[[74  1]
 [ 6 19]]
Test Precision:  0.95 
 Test Recall:  0.76 
 F1-score: 0.8444444444444444
Runing Child: 1
Using sequence_length= 25
Accurracy: 0.974243016120951
18131/18131 [==============================] - 7s 381us/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels
[[14891   140]
 [  327  2773]]
training_precision =  0.9519395811877789 
 training_recall =  0.8945161290322581
(100, 25, 25)
(100, 1)
Accurracy: 0.97
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels
[[74  1]
 [ 2 23]]
Test Precision:  0.9583333333333334 


### Get metrics from Azure ML Experiment

In [30]:
metrics = root_run.get_metrics(recursive=True)
metrics

{'23cc3166-fe96-4d0a-81bc-cb3a3849fcb7': {'Test_F1Score': 0.8444444444444444,
  'Test_Recall': 0.76,
  'Test_Precision': 0.95,
  'DROPOUT': 0.2,
  'LSTM_2': 50,
  'LSTM_1': 100,
  'Epochs': 10,
  'sequence_length': 10},
 '30bed365-f281-4673-8937-b86f819a26bd': {'Test_F1Score': 0.9387755102040817,
  'Test_Recall': 0.92,
  'Test_Precision': 0.9583333333333334,
  'DROPOUT': 0.2,
  'LSTM_2': 50,
  'LSTM_1': 100,
  'Epochs': 10,
  'sequence_length': 25},
 '7982050f-ed3a-40a9-9f5c-a4c4c6b6156c': {'Test_F1Score': 0.9411764705882353,
  'Test_Recall': 0.96,
  'Test_Precision': 0.9230769230769231,
  'DROPOUT': 0.2,
  'LSTM_2': 50,
  'LSTM_1': 100,
  'Epochs': 10,
  'sequence_length': 50},
 '56bb3a41-17bb-43a9-8d2c-2d806a3d4063': {'Test_F1Score': 0.9411764705882353,
  'Test_Recall': 0.96,
  'Test_Precision': 0.9230769230769231,
  'DROPOUT': 0.2,
  'LSTM_2': 50,
  'LSTM_1': 100,
  'Epochs': 10,
  'sequence_length': 70}}

In [31]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}

for r in root_run.get_children():
    child_runs[r.id] = r

In [32]:
best_run_id = max(metrics, key = lambda k: metrics[k]['Test_F1Score'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Test_F1Score:', metrics[best_run_id]['Test_F1Score'])
print('Test_Precision:', metrics[best_run_id]['Test_Precision'])
print('Test_Recall:', metrics[best_run_id]['Test_Recall'])
print("sequence_length=", metrics[best_run_id]['sequence_length'])

Best run is: 7982050f-ed3a-40a9-9f5c-a4c4c6b6156c
Test_F1Score: 0.9411764705882353
Test_Precision: 0.9230769230769231
Test_Recall: 0.96
sequence_length= 50


### Register best model and scaler as a MODELS in Azure ML Service Workspace

In [42]:
#Register the Keras model directly from RUN object
#azmodelname = model_name.split(".")[0]
#azmodel = best_run.register_model(azmodelname, model_path="/outputs/"+model_name, tags={"F1Score":metrics[best_run_id]['Test_F1Score'], "sequence_length":metrics[best_run_id]['sequence_length']} )

In [43]:
#azmodelname

In [33]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Predictive_Maintenance,7982050f-ed3a-40a9-9f5c-a4c4c6b6156c,,Completed,Link to Azure Portal,Link to Documentation


In [34]:
#Register the Min Max model from local file

from azureml.core.model import Model
azmodelminmax = Model.register(model_path = scaler_filename, # this points to a local file
                       model_name = scaler_filename, # this is the name the model is registered as, am using same name for both path and name.                 
                       description = "Scikit MINMAX scaler for Predictve Maintance model",
                       workspace = ws)

print(azmodelminmax.name, azmodelminmax.description, azmodelminmax.version)

Registering model scaler.pkl
scaler.pkl Scikit MINMAX scaler for Predictve Maintance model 1


In [35]:
# Find the best run
best_run.get_file_names()

['outputs/predictivemodel.h5']

In [36]:
best_model_name = 'best_run_dnld_predictivemodel.h5'

In [37]:
best_run.download_file(name='outputs/predictivemodel.h5', output_file_path='best_run_dnld_predictivemodel.h5')

In [38]:
azmodelbestmodel = Model.register(model_path = 'best_run_dnld_predictivemodel.h5',
                       model_name = 'best_run_dnld_predictivemodel.h5',
                       tags = {'type': "lstm", 'target': "ATA_CODE"},
                       description = "LSTM model for predictive maintenance",
                       workspace = ws)

Registering model best_run_dnld_predictivemodel.h5


## Deployment of Model

In [39]:
from sklearn.externals import joblib
import json
from keras.models import load_model

def init():
    # read in the model file
    global model
    global min_max_scaler
    
    # load model
    model = load_model(best_model_name)
    print("Model Loaded")
        
    # And now to load...
    min_max_scaler = joblib.load(scaler_filename) 
    print("Scaler Loaded")


In [40]:
def run(input_json):
    try:
        data = json.loads(input_json)['data']
        data = pd.read_json(data, orient='records')
        
        # Normalize data
        cols_normalize = data.columns.difference(['id','cycle'])
        print(cols_normalize)
        data['cycle_norm'] = data['cycle']
        norm_data = pd.DataFrame(min_max_scaler.transform(data[cols_normalize]), 
                                    columns=cols_normalize, 
                                    index=data.index)
        data_join_df = data[data.columns.difference(cols_normalize)].join(norm_data)
        data = data_join_df.reindex(columns = data.columns)
        data = data.reset_index(drop=True)
        data.head()
        
        # pick the feature columns 
        sensor_cols = ['s' + str(i) for i in range(1,22)]
        sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
        sequence_cols.extend(sensor_cols)
        
        # sequence_length
        sequence_length = 50
        
        # Transforming Input Set
        seq_array_test_last = [data[data['id']==id][sequence_cols].values[-sequence_length:] 
                           for id in data['id'].unique() if len(data[data['id']==id]) >= sequence_length]

        seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
        seq_array_test_last.shape

        y_mask = [len(data[data['id']==id]) >= sequence_length for id in data['id'].unique()]

        print(seq_array_test_last.shape)
        
        # make predictions and compute confusion matrix
        y_pred_test = model.predict_classes(seq_array_test_last)
      
        # Send results
        pred = y_pred_test.tolist()
        return json.dumps({"result": pred})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

In [41]:
init()

Model Loaded
Scaler Loaded


In [42]:
run(json.dumps({"data": test_df_original.to_json(orient='records')}))

Index(['cycle_norm', 's1', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       's17', 's18', 's19', 's2', 's20', 's21', 's3', 's4', 's5', 's6', 's7',
       's8', 's9', 'setting1', 'setting2', 'setting3'],
      dtype='object')
(93, 50, 25)


'{"result": [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]}'

In [43]:
MODEL_NAME = 'best_run_dnld_predictivemodel.h5'

In [44]:
#%%writefile score_script.py
score_script = """

import pandas as pd
import numpy as np
from sklearn.externals import joblib
import json
from keras.models import load_model
from azureml.core.model import Model

def init():
    # read in the model file
    global model
    global min_max_scaler
    
    # load model
    MODEL_NAME = 'best_run_dnld_predictivemodel.h5' #interpolated
    model_path = Model.get_model_path(model_name = MODEL_NAME)
    model = load_model(model_path)
    print("Model Loaded")
        
    # And now to load...
    model_name = 'scaler.pkl' #interpolated
    model_path = Model.get_model_path(model_name = model_name)
    min_max_scaler = joblib.load(model_path) 
    print("Scaler Loaded")

    
def run(input_json):
    try:
        data = json.loads(input_json)['data']
        data = pd.read_json(data, orient='records')
        
        # Normalize data
        cols_normalize = data.columns.difference(['id','cycle'])
        print(cols_normalize)
        data['cycle_norm'] = data['cycle']
        norm_data = pd.DataFrame(min_max_scaler.transform(data[cols_normalize]), 
                                    columns=cols_normalize, 
                                    index=data.index)
        data_join_df = data[data.columns.difference(cols_normalize)].join(norm_data)
        data = data_join_df.reindex(columns = data.columns)
        data = data.reset_index(drop=True)
        data.head()
        
        # pick the feature columns 
        sensor_cols = ['s' + str(i) for i in range(1,22)]
        sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
        sequence_cols.extend(sensor_cols)
        
        # sequence_length
        sequence_length = 50
        
        # Transforming Input Set
        seq_array_test_last = [data[data['id']==id][sequence_cols].values[-sequence_length:] 
                           for id in data['id'].unique() if len(data[data['id']==id]) >= sequence_length]

        seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
        seq_array_test_last.shape

        y_mask = [len(data[data['id']==id]) >= sequence_length for id in data['id'].unique()]

        print(seq_array_test_last.shape)
        
        # make predictions and compute confusion matrix
        y_pred_test = model.predict_classes(seq_array_test_last)
      
        # Send results
        pred = y_pred_test.tolist()
        return json.dumps({{"result": pred}})
    except Exception as e:
        result = str(e)
        return json.dumps({{"error": result}})
    
""".format(model_name=MODEL_NAME, scaler_filename=scaler_filename)

exec(score_script)

with open("score_script.py", "w") as file:
    file.write(score_script)

In [47]:
%%sh 
cat score_script.py



import pandas as pd
import numpy as np
from sklearn.externals import joblib
import json
from keras.models import load_model
from azureml.core.model import Model

def init():
    # read in the model file
    global model
    global min_max_scaler
    
    # load model
    MODEL_NAME = 'best_run_dnld_predictivemodel.h5' #interpolated
    model_path = Model.get_model_path(model_name = MODEL_NAME)
    model = load_model(model_path)
    print("Model Loaded")
        
    # And now to load...
    model_name = 'scaler.pkl' #interpolated
    model_path = Model.get_model_path(model_name = model_name)
    min_max_scaler = joblib.load(model_path) 
    print("Scaler Loaded")

    
def run(input_json):
    try:
        data = json.loads(input_json)['data']
        data = pd.read_json(data, orient='records')
        
        # Normalize data
        cols_normalize = data.columns.difference(['id','cycle'])
        print(cols_normalize)
        data['cycle_norm'] = data['cycle']
        norm_data = 

### Create Conda file for the docker image creation

In [48]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas', 'keras', 'tensorflow', 'h5py']) #showing how to add libs as an example - not needed for this model.

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

In [49]:
%%sh 
cat mydeployenv.yml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
- scikit-learn
- numpy
- pandas
- keras
- tensorflow
- h5py


## Create ACI config

In [50]:
#deploy to ACI
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':model_name}, 
    description = 'Predictive Maintenance model')

### Create docker image and web service on the same call

In [51]:
%%time
service_name = "aci4-predictive-maint"
runtime = "python" 
driver_file = "score_script.py"
my_conda_file = "mydeployenv.yml"

# image creation
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                    runtime = runtime, 
                                    conda_file = my_conda_file)



image = ContainerImage.create(name = "predictivemaintenance" + ".image",
                              # this is the model object
                              models = [azmodelbestmodel, azmodelminmax],
                              image_config = image_config,
                              workspace = ws)
image.wait_for_creation(show_output = True)

# # Webservice creation
# myservice = Webservice.deploy_from_model(
#   workspace=ws, 
#   name=service_name,
#   deployment_config = myaci_config,
#   models = [azmodel, azmodelminmax],
#   image_config = myimage_config
#     )

# myservice.wait_for_deployment(show_output=True)

Creating image
Running.......................................................................
SucceededImage creation operation finished for image predictivemaintenance.image:1, operation "Succeeded"
CPU times: user 2.72 s, sys: 572 ms, total: 3.29 s
Wall time: 6min 34s


In [60]:
# Azure Container Service (ACI) Name
ACI_SERVICE_NAME = "jptrpredmaint" + '-aciservice'

# Azure Kubernetes Service (AKS) Name
AKS_SERVICE_NAME = "predmaint" + '-aksservice'

In [61]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

aci_service_name = ACI_SERVICE_NAME.lower()

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 4, 
                                               tags = {'type': "lstm", 'target': "maintenance"}, 
                                               description = "LSTM model for predictive maintenance")

In [62]:
#Let's see if we have an ACI web service already running in the workspace
aci_service = ""
for aci in AciWebservice.list(workspace=ws):
    if (aci.compute_type == "ACI"):
        if (aci.name == aci_service_name): 
            aci_service = aci
            print("Existing ACI Service name:", aci_service.name)
        else:
            print("No service by the name of **"+aci_service_name+"** exists!")

No service by the name of **jptrpredmaint-aciservice** exists!


In [63]:
%%time
# We update the image if service exists or create a new service if doesnt exist
if (aci_service == ""):
    aci_service = AciWebservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
    aci_service.wait_for_deployment(True)
    print(aci_service.state)
else:
    aci_service.update(image=image)
    aci_service.wait_for_deployment(True)
    print(aci_service.state)

Creating service
Running...................................................................................................................................................................................................................................
TimedOutACI service creation operation finished, operation "TimedOut"
Service creation polling reached terminal state, current service state: Transitioning
Service creation polling reached terminal state, unexpected response received.
Transitioning
CPU times: user 7.96 s, sys: 1.74 s, total: 9.7 s
Wall time: 20min 14s


In [ ]:
print(aci_service.get_logs())

##TEST Scoring

In [ ]:
import urllib
import requests
import json

print(aci_service.scoring_uri)
# The URL will need to be editted after service create.
url_aci = aci_service.scoring_uri

headers = {'Content-Type':'application/json'}

body = json.dumps({"data": test_df_original.to_json(orient='records')})

#Send Request to ACI service and print response
req_aci = urllib.request.Request(url_aci, str.encode(body), headers) 
print(urllib.request.urlopen(req_aci).read())